In [1]:
import numpy as np
from astropy.io import fits
from astropy.table import Table as Table
import matplotlib.pyplot as plt
from linetools.spectra import utils as ltsu
from linetools.spectra.xspectrum1d import XSpectrum1D
import astropy.units as u
from astropy import constants as const
from numpy import random as ran
%matplotlib 

Using matplotlib backend: Qt5Agg


# the same routine as stack.py

In [2]:
#the 2016 survey

#to order and match the spec with their z values
spec_atr_16 = Table.read('/home/jsm/PycharmProjects/tau_eff/Spectra/2016/spec_atr.txt', format = 'ascii') 

#an initial filter to ensure the spec included cover lya
basestring_16 = '/home/jsm/PycharmProjects/tau_eff/Spectra/2016/'

init_spec_16 = []

init_z_16 = []
    
for entry in spec_atr_16:
    
    if 2.25 < entry["zspec"] < 2.5: #the redshift bin
        
        if entry["Conf"] < 10.0:  # excluding any QSOs
        
            temp = XSpectrum1D.from_file(basestring_16 + entry["Filename"])
        
            if temp.wvmin < (1216*u.AA)*(1+entry["zspec"]) < temp.wvmax:
                
                init_z_16.append(entry["zspec"])
                
                init_spec_16.append(XSpectrum1D.from_file(basestring_16 + entry["Filename"]))

In [3]:
#the 2017 survey

#to order and match the spec with their z values
spec_atr_17 = Table.read('/home/jsm/PycharmProjects/tau_eff/Spectra/2017/spec_atr.txt',
                         format = 'ascii') 

#an initial filter to ensure the spec included cover lya
basestring_17 = '/home/jsm/PycharmProjects/tau_eff/Spectra/2017/'

init_spec_17 = []

init_z_17 = []
    
for entry in spec_atr_17:
    
    if 2.25 < entry["col5"] < 2.5: #the redshift bin
        
        if entry["col4"] < 10.0:

            temp = XSpectrum1D.from_file(basestring_17 + entry["col1"])
        
            if temp.wvmin < (1216*u.AA)*(1+entry["col5"]) < temp.wvmax:
                
                init_z_17.append(entry["col5"])
                
                init_spec_17.append(XSpectrum1D.from_file(basestring_17 + entry["col1"]))

/home/jsm/.pyenv/versions/anaconda3-2.4.1/lib/python3.5/site-packages/astropy/units/quantity.py:639: RuntimeWarning: invalid value encountered in greater
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)


In [4]:
spec = np.asarray(init_spec_16 + init_spec_17)

print("Number of spectra (Nspec) after filter =", len(spec))

red = np.asarray(init_z_16 + init_z_17)

Number of spectra (Nspec) after filter = 214


In [6]:
r = range(len(spec))

s2n = 2.5

temp = [np.asarray(spec[i].wavelength / (1 + red[i])) for i in r]  # rest-frame

norm = [(1260 < entry) & (entry < 1304) for entry in temp]

signal = np.asarray([spec[i].flux[norm[i]] for i in r])

noise = np.asarray([spec[i].sig[norm[i]] for i in r])

s2n_array = np.asarray([np.median(signal[i] / noise[i]) for i in r])

s2n_cut = [i > s2n for i in s2n_array]

spec = spec[s2n_cut]

red = red[s2n_cut]

z = np.median(red)

In [7]:
r = range(len(spec))

temp = [np.asarray(spec[i].wavelength / (1 + red[i])) for i in r]

wv_norm = [(1260 < entry) & (entry < 1304) for entry in temp]  # just between the SiII lines.

flux_range = np.asarray([spec[i].flux[wv_norm[i]] for i in r])

medians = np.asarray([np.median(flux_range[i]) for i in r])

norm_flux = np.asarray([(spec[i].flux / medians[i]) for i in r])

norm_spec = [XSpectrum1D(spec[i].wavelength, norm_flux[i], spec[i].sig) for i in r]  # the normalized spec array

    # truncating each spectra to only include wavelength values from 950A to 1500A

/home/jsm/.pyenv/versions/anaconda3-2.4.1/lib/python3.5/site-packages/linetools-0.3.dev2065-py3.5.egg/linetools/spectra/xspectrum1d.py:254: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")


In [8]:
r = range(len(norm_spec))

temp = np.array([norm_spec[i].wavelength / (1 + red[i]) for i in r])

trim = []

for i in r:
    trim.append([950 * u.AA <= temp[i][j] <= 1500 * u.AA for j in range(len(temp[i]))])
        # to include only enough to extrapolate from

t_spec = []

for i in r:
    t_spec.append(XSpectrum1D(norm_spec[i].wavelength[trim[i]][0:935],
                              norm_spec[i].flux[trim[i]][0:935],
                              norm_spec[i].flux[trim[i]][0:935]))

/home/jsm/.pyenv/versions/anaconda3-2.4.1/lib/python3.5/site-packages/linetools-0.3.dev2065-py3.5.egg/linetools/spectra/xspectrum1d.py:254: UserWarning: No unit given to wavelength, assuming Angstroms.
  warnings.warn("No unit given to wavelength, assuming Angstroms.")


In [9]:
collate = ltsu.collate(t_spec)  # to get a single XSpec object

fix = []  # to make sure none of the values included will mess up the stack

a = collate.data['wave']

for i in range(collate.nspec):

    if np.min(a[i]) == 0.0:
        fix.append(i)

if len(fix) > 0:

    print("deleting these spectra", fix)

    good_spec = np.delete(t_spec,fix) # exclude all spec[fix]

    red = np.delete(red,fix)
    
print("Number of spectra to be stacked =", collate.nspec)

collate = ltsu.collate(good_spec)


deleting these spectra [44, 70]
Number of spectra to be stacked = 92


In [13]:
rest_spec = ltsu.rebin_to_rest(collate, red, 300 * u.km / u.s, grow_bad_sig=True)

In [11]:
stack = ltsu.smash_spectra(rest_spec)

stack.plot()

# Example Spec

In [26]:
plt.figure(figsize=(10, 5))

plt.plot(spec[43].wavelength,spec[43].flux,color="black",label="LBG (z~2.45) ")
plt.plot(spec[43].wavelength,spec[43].sig, color="grey",label="Flux Error")

plt.hlines(0,3500,5000, color="#fd8d3c",linestyle="--")

plt.text(4215,.25,"$Lyα$",fontsize=12,color="#fd8d3c")

plt.xlabel("$\lambda_{obs}$",fontsize=15)
plt.ylabel("Relative flux",fontsize=15)

plt.legend(fontsize=12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.xlim(3500,5000)
plt.ylim(-.05,.45)
plt.show()

In [20]:
plt.fill_between?